In [ ]:
# import libraries
from google.colab import files
import pandas as pd
import numpy as np
from sklearn import metrics
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
# pycaret
!pip install pycaret
import pycaret as pc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  

In [ ]:
from pycaret.classification import *

In [ ]:
#from pc.utils import enable_colab
#enable_colab()

# ERROR IN ABOVE - BUT NOT NECESSARY FOR MODELING

In [ ]:
# load Cleveland datafile
cleveland_csv_path = "/processed.cleveland.data"
cleveland_data = pd.read_csv(cleveland_csv_path, header = None)
# assign column names
cleveland_data.set_axis(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs',
                  'restecg', 'thalach', 'exang', 'oldpeak', 'slope',
                  'ca', 'thal', 'num'], axis = 1, inplace = True)

In [ ]:
# Define label column
heart_label = 'num'

# SLIGHTLY BETTER MODEL
no missing values & stratified sampling

## DATA PREPROCESSING

### MISSING VALUES

In [ ]:
# make copy of original dataframe
cleveland = cleveland_data.copy()

In [ ]:
# check for missing values
cleveland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    float64
 1   sex       303 non-null    float64
 2   cp        303 non-null    float64
 3   trestbps  303 non-null    float64
 4   chol      303 non-null    float64
 5   fbs       303 non-null    float64
 6   restecg   303 non-null    float64
 7   thalach   303 non-null    float64
 8   exang     303 non-null    float64
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    float64
 11  ca        303 non-null    object 
 12  thal      303 non-null    object 
 13  num       303 non-null    int64  
dtypes: float64(11), int64(1), object(2)
memory usage: 33.3+ KB


In [ ]:
# replace '?' values with NaN so can impute
cleveland['thal'].replace('?', np.NaN, inplace=True)
cleveland['thal'] = cleveland['thal'].astype(float)
cleveland['ca'].replace('?', np.NaN, inplace=True)
cleveland['ca'] = cleveland['ca'].astype(float)
cleveland.tail() # to verify didn't mess up dataframe IDs

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,2
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,3
301,57.0,0.0,2.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,2.0,1.0,3.0,1
302,38.0,1.0,3.0,138.0,175.0,0.0,0.0,173.0,0.0,0.0,1.0,NaN,3.0,0


In [ ]:
from sklearn.impute import SimpleImputer
# impute with mode as 'thal' and 'ca' (attributes w/ missing values) are discrete
imputeMode = SimpleImputer(strategy="most_frequent") # create mode imputer
imputeMode.fit(cleveland) # fit - learns the data
imputed = imputeMode.transform(cleveland) # transform - imputes with chosen strategy
cleveland = pd.DataFrame(imputed, columns=cleveland.columns, index=cleveland['thal'].index) # back to pandas DataFrame
cleveland.info() # check for missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    float64
 1   sex       303 non-null    float64
 2   cp        303 non-null    float64
 3   trestbps  303 non-null    float64
 4   chol      303 non-null    float64
 5   fbs       303 non-null    float64
 6   restecg   303 non-null    float64
 7   thalach   303 non-null    float64
 8   exang     303 non-null    float64
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    float64
 11  ca        303 non-null    float64
 12  thal      303 non-null    float64
 13  num       303 non-null    float64
dtypes: float64(14)
memory usage: 33.3 KB


In [ ]:
cleveland.head() # check for anything obviously wonky

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0.0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2.0
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1.0
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0.0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0.0


In [ ]:
cleveland.corr()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
age,1.000000,-0.097542,0.104139,0.284946,0.208950,0.118530,0.148868,-0.393806,0.091661,0.203805,0.161770,0.365323,0.128303,0.223120
sex,-0.097542,1.000000,0.010084,-0.064456,-0.199915,0.047862,0.021647,-0.048663,0.146201,0.102173,0.037533,0.086048,0.380581,0.276816
cp,0.104139,0.010084,1.000000,-0.036077,0.072319,-0.039975,0.067505,-0.334422,0.384060,0.202277,0.152050,0.233117,0.262089,0.414446
trestbps,0.284946,-0.064456,-0.036077,1.000000,0.130120,0.175340,0.146560,-0.045351,0.064762,0.189171,0.117382,0.097528,0.134424,0.150825
chol,0.208950,-0.199915,0.072319,0.130120,1.000000,0.009841,0.171043,-0.003432,0.061310,0.046564,-0.004062,0.123726,0.018351,0.085164
fbs,0.118530,0.047862,-0.039975,0.175340,0.009841,1.000000,0.069564,-0.007854,0.025665,0.005747,0.059894,0.140764,0.064625,0.025264
restecg,0.148868,0.021647,0.067505,0.146560,0.171043,0.069564,1.000000,-0.083389,0.084867,0.114133,0.133946,0.131749,0.024325,0.169202
thalach,-0.393806,-0.048663,-0.334422,-0.045351,-0.003432,-0.007854,-0.083389,1.000000,-0.378103,-0.343085,-0.385601,-0.265699,-0.274142,-0.417167
exang,0.091661,0.146201,0.384060,0.064762,0.061310,0.025665,0.084867,-0.378103,1.000000,0.288223,0.257748,0.145788,0.325240,0.431894
oldpeak,0.203805,0.102173,0.202277,0.189171,0.046564,0.005747,0.114133,-0.343085,0.288223,1.000000,0.577537,0.301067,0.342405,0.424510


In [ ]:
cleveland.drop(['slope', 'cp', 'thal', 'fbs', 'restecg'], axis=1, inplace=True)

### BINARY LABEL
(multiclass for "future research")

In [ ]:
cleveland['num'][cleveland['num'] > 0] = 1
cleveland.head()

,age,sex,trestbps,chol,thalach,exang,oldpeak,ca,num
0,63.0,1.0,145.0,233.0,150.0,0.0,2.3,0.0,0.0
1,67.0,1.0,160.0,286.0,108.0,1.0,1.5,3.0,1.0
2,67.0,1.0,120.0,229.0,129.0,1.0,2.6,2.0,1.0
3,37.0,1.0,130.0,250.0,187.0,0.0,3.5,0.0,0.0
4,41.0,0.0,130.0,204.0,172.0,0.0,1.4,0.0,0.0


### STRATIFIED SAMPLING

(due to difference sex representation)

In [ ]:
cleveland_strat = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in cleveland_strat.split(cleveland, cleveland["num"]):
  cleveland_strat_train = cleveland.loc[train_index]
  cleveland_strat_test = cleveland.loc[test_index]

In [ ]:
cleveland_strat_test["num"].value_counts()/len(cleveland_strat_test)

0.0    0.540984
1.0    0.459016
Name: num, dtype: float64

In [ ]:
cleveland_strat_train["num"].value_counts()/len(cleveland_strat_train)

0.0    0.541322
1.0    0.458678
Name: num, dtype: float64

Less significant for Cleveland and Statlog datasets, which have less dramatic difference in sex representation in data, but different for other three datasets. 

In [ ]:
# split stratified data
cleveland_strat_train_X = cleveland_strat_train.drop([heart_label], axis=1)
cleveland_strat_train_y = cleveland_strat_train[heart_label]
cleveland_strat_test_X = cleveland_strat_test.drop([heart_label], axis=1)
cleveland_strat_test_y = cleveland_strat_test[heart_label]
cleveland_strat_train_X.sample(5)

,age,sex,trestbps,chol,thalach,exang,oldpeak,ca
98,52.0,1.0,134.0,201.0,158.0,0.0,0.8,1.0
207,50.0,1.0,144.0,200.0,126.0,1.0,0.9,0.0
8,63.0,1.0,130.0,254.0,147.0,0.0,1.4,1.0
13,44.0,1.0,120.0,263.0,173.0,0.0,0.0,0.0
12,56.0,1.0,130.0,256.0,142.0,1.0,0.6,1.0


## Correlated Block

In [ ]:
# set up for correlated
cleveland_pc_setup = setup(data = cleveland_strat_train, test_data = cleveland_strat_test, target = 'num', session_id=42)

,Description,Value
0,Session id,42
1,Target,num
2,Target type,Binary
3,Original data shape,"(303, 9)"
4,Transformed data shape,"(303, 9)"
5,Transformed train set shape,"(242, 9)"
6,Transformed test set shape,"(61, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


In [ ]:
# compare models
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7975,0.8710,0.7402,0.8172,0.7681,0.5901,0.6008,0.5230
ridge,Ridge Classifier,0.7892,0.0000,0.7038,0.8248,0.7506,0.5712,0.5850,0.0450
et,Extra Trees Classifier,0.7848,0.8643,0.7576,0.7778,0.7637,0.5668,0.5715,0.3910
lda,Linear Discriminant Analysis,0.7808,0.8735,0.6947,0.8162,0.7427,0.5547,0.5674,0.0570
nb,Naive Bayes,0.7767,0.8651,0.7303,0.7935,0.7524,0.5500,0.5603,0.0520
rf,Random Forest Classifier,0.7720,0.8374,0.7023,0.7820,0.7360,0.5371,0.5435,0.4090
qda,Quadratic Discriminant Analysis,0.7642,0.8493,0.7030,0.7770,0.7330,0.5230,0.5299,0.0570
gbc,Gradient Boosting Classifier,0.7598,0.8306,0.7295,0.7430,0.7338,0.5155,0.5183,0.2730
lightgbm,Light Gradient Boosting Machine,0.7557,0.8322,0.6939,0.7652,0.7224,0.5056,0.5135,0.1660
xgboost,Extreme Gradient Boosting,0.7475,0.8171,0.7023,0.7447,0.7161,0.4896,0.4973,0.0980


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
# create model
lr = create_model('lr')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7600,0.8654,0.5833,0.8750,0.7000,0.5130,0.5424
1,0.8400,0.9286,0.8182,0.8182,0.8182,0.6753,0.6753
2,0.6667,0.6923,0.6364,0.6364,0.6364,0.3287,0.3287
3,0.8750,0.8531,0.8182,0.9000,0.8571,0.7465,0.7492
4,0.8750,0.9580,0.8182,0.9000,0.8571,0.7465,0.7492
5,0.7500,0.8601,0.7273,0.7273,0.7273,0.4965,0.4965
6,0.7917,0.9720,0.5455,1.0000,0.7059,0.5652,0.6276
7,0.8333,0.8671,0.9091,0.7692,0.8333,0.6690,0.6783
8,0.8333,0.8811,0.8182,0.8182,0.8182,0.6643,0.6643


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# tune model
tuned_lr = tune_model(lr)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8400,0.8526,0.7500,0.9000,0.8182,0.6774,0.6864
1,0.8400,0.9351,0.8182,0.8182,0.8182,0.6753,0.6753
2,0.6250,0.6993,0.6364,0.5833,0.6087,0.2500,0.2509
3,0.8750,0.8462,0.8182,0.9000,0.8571,0.7465,0.7492
4,0.8750,0.9441,0.8182,0.9000,0.8571,0.7465,0.7492
5,0.7500,0.8671,0.7273,0.7273,0.7273,0.4965,0.4965
6,0.8333,0.9790,0.6364,1.0000,0.7778,0.6547,0.6976
7,0.8333,0.8741,0.9091,0.7692,0.8333,0.6690,0.6783
8,0.8333,0.8741,0.8182,0.8182,0.8182,0.6643,0.6643


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [ ]:
predict_model(tuned_lr)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.8525,0.9383,0.8929,0.8065,0.8475,0.7053,0.7087


,age,sex,trestbps,chol,thalach,exang,oldpeak,ca,num,prediction_label,prediction_score
219,59.0,1.0,138.0,271.0,182.0,0.0,0.0,0.0,0.0,0,0.8540
271,66.0,1.0,160.0,228.0,138.0,0.0,2.3,0.0,0.0,1,0.5821
89,51.0,0.0,130.0,256.0,149.0,0.0,0.5,0.0,0.0,0,0.9039
101,34.0,1.0,118.0,182.0,174.0,0.0,0.0,0.0,0.0,0,0.8577
67,54.0,1.0,150.0,232.0,165.0,0.0,1.6,0.0,0.0,0,0.6569
...,...,...,...,...,...,...,...,...,...,...,...
285,58.0,1.0,114.0,318.0,140.0,0.0,4.4,3.0,1.0,1,0.9892
243,61.0,1.0,134.0,234.0,145.0,0.0,2.6,2.0,1.0,1,0.9148
94,63.0,0.0,135.0,252.0,172.0,0.0,0.0,0.0,0.0,0,0.9655
291,55.0,0.0,132.0,342.0,166.0,0.0,1.2,0.0,0.0,0,0.8965


## BINARY MODEL & Uncorreleated (do not execute to keep printed..)

In [ ]:
# set up
cleveland_pc_setup = setup(data = cleveland_strat_train, test_data = cleveland_strat_test, target = 'num', session_id=42)

,Description,Value
0,Session id,42
1,Target,num
2,Target type,Binary
3,Original data shape,"(303, 14)"
4,Transformed data shape,"(303, 14)"
5,Transformed train set shape,"(242, 14)"
6,Transformed test set shape,"(61, 14)"
7,Numeric features,13
8,Preprocess,True
9,Imputation type,simple


In [ ]:
# compare models
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.8257,0.8906,0.7659,0.8581,0.8037,0.6479,0.6578,0.0670
ridge,Ridge Classifier,0.8215,0.0000,0.7659,0.8499,0.7998,0.6396,0.6493,0.0830
nb,Naive Bayes,0.8180,0.8907,0.7659,0.8298,0.7939,0.6316,0.6367,0.0730
lightgbm,Light Gradient Boosting Machine,0.8175,0.8581,0.7841,0.8295,0.8004,0.6328,0.6399,0.2880
lr,Logistic Regression,0.8173,0.8857,0.7659,0.8392,0.7952,0.6311,0.6408,0.3980
rf,Random Forest Classifier,0.8012,0.8794,0.7394,0.8336,0.7755,0.5984,0.6102,0.4120
xgboost,Extreme Gradient Boosting,0.7888,0.8501,0.7667,0.7832,0.7712,0.5757,0.5802,0.0990
gbc,Gradient Boosting Classifier,0.7885,0.8469,0.7659,0.7871,0.7712,0.5752,0.5814,0.1460
et,Extra Trees Classifier,0.7885,0.8875,0.7386,0.7991,0.7639,0.5729,0.5788,0.2190
qda,Quadratic Discriminant Analysis,0.7847,0.8670,0.7568,0.7856,0.7662,0.5671,0.5729,0.0670


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

LinearDiscriminantAnalysis(covariance_estimator=None, n_components=None,
                           priors=None, shrinkage=None, solver='svd',
                           store_covariance=False, tol=0.0001)

In [ ]:
# create model
lda = create_model('lda')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8800,0.8910,0.7500,1.0000,0.8571,0.7573,0.7806
1,0.9600,0.9870,0.9091,1.0000,0.9524,0.9180,0.9211
2,0.7500,0.7762,0.6364,0.7778,0.7000,0.4894,0.4966
3,0.8750,0.8741,0.8182,0.9000,0.8571,0.7465,0.7492
4,0.7500,0.9161,0.7273,0.7273,0.7273,0.4965,0.4965
5,0.9167,0.9231,0.9091,0.9091,0.9091,0.8322,0.8322
6,0.8750,0.9860,0.7273,1.0000,0.8421,0.7429,0.7687
7,0.6667,0.8671,0.8182,0.6000,0.6923,0.3469,0.3671
8,0.8333,0.8951,0.7273,0.8889,0.8000,0.6596,0.6693


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# tune model
tuned_lda = tune_model(lda)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8800,0.9167,0.7500,1.0000,0.8571,0.7573,0.7806
1,0.9600,0.9870,0.9091,1.0000,0.9524,0.9180,0.9211
2,0.7500,0.7692,0.6364,0.7778,0.7000,0.4894,0.4966
3,0.8333,0.8811,0.8182,0.8182,0.8182,0.6643,0.6643
4,0.7917,0.9161,0.7273,0.8000,0.7619,0.5775,0.5795
5,0.9167,0.9371,0.9091,0.9091,0.9091,0.8322,0.8322
6,0.8333,0.9790,0.6364,1.0000,0.7778,0.6547,0.6976
7,0.7917,0.8881,0.9091,0.7143,0.8000,0.5890,0.6078
8,0.8333,0.9091,0.7273,0.8889,0.8000,0.6596,0.6693


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [ ]:
# display tuned model
print(tuned_lda)

LinearDiscriminantAnalysis(covariance_estimator=None, n_components=None,
                           priors=None, shrinkage='auto', solver='lsqr',
                           store_covariance=False, tol=0.0001)


In [ ]:
predict_model(tuned_lda)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Linear Discriminant Analysis,0.8525,0.9524,0.8929,0.8065,0.8475,0.7053,0.7087


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,prediction_label,prediction_score
219,59.0,1.0,4.0,138.0,271.0,0.0,2.0,182.0,0.0,0.0,1.0,0.0,3.0,0.0,0,0.9149
271,66.0,1.0,4.0,160.0,228.0,0.0,2.0,138.0,0.0,2.3,1.0,0.0,6.0,0.0,1,0.7858
89,51.0,0.0,3.0,130.0,256.0,0.0,2.0,149.0,0.0,0.5,1.0,0.0,3.0,0.0,0,0.9802
101,34.0,1.0,1.0,118.0,182.0,0.0,2.0,174.0,0.0,0.0,1.0,0.0,3.0,0.0,0,0.9937
67,54.0,1.0,3.0,150.0,232.0,0.0,2.0,165.0,0.0,1.6,1.0,0.0,7.0,0.0,1,0.5419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,58.0,1.0,4.0,114.0,318.0,0.0,1.0,140.0,0.0,4.4,3.0,3.0,6.0,1.0,1,0.9980
243,61.0,1.0,1.0,134.0,234.0,0.0,0.0,145.0,0.0,2.6,2.0,2.0,3.0,1.0,0,0.6571
94,63.0,0.0,3.0,135.0,252.0,0.0,2.0,172.0,0.0,0.0,1.0,0.0,3.0,0.0,0,0.9883
291,55.0,0.0,2.0,132.0,342.0,0.0,0.0,166.0,0.0,1.2,1.0,0.0,3.0,0.0,0,0.9940
